In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN_co.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 1/1 [00:00<00:00, 5722.11it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 1/1 [00:00<00:00, 260.02it/s]

Standard deviation exceeded 1e-16 tolerance: 1.9894890166455426e-15, for i_q:0


In [6]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]
l_qpts = np.array(epsinp.qpts)
l_qpts[0] *= 0
qpts = QPoints.from_cryst(wfndata.kpts.recilat, None, *l_qpts)


In [7]:
kernelclass = KernelMtxEl(
    crystal = wfndata.crystal,
    gspace = wfndata.grho,
    kpts = wfndata.kpts,
    l_wfn = wfndata.l_wfn,
    l_gsp_wfn =  wfndata.l_gk,
    qpts = qpts, 
    epsinp = epsinp,
    sigmainp = sigmainp,
    l_epsmats = epsilon.l_epsinv,
    q0val=q0val,
    )

In [8]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [9]:
data = kernelclass.calc_kernel_mtxel()

0


In [10]:
exc = data["exc"]
print(f"The exc kernel is {exc}")

head = data["head"]
print(f"The head kernel is {head}")

wings = data["wings"]
print(f"The wings kernel is {wings}")

body = data["body"]
print(f"The body kernel is {body}")

The exc kernel is [[[[[[0.39326425+2.77555756e-17j]]]]]]
The head kernel is [[[[[[1.-1.89016773e-19j]]]]]]
The wings kernel is [[[[[[0.+0.j]]]]]]
The body kernel is [[[[[[-0.04643365+7.95732181e-07j]]]]]]


In [11]:
l_epsinv = epsilon.l_epsinv
eps0mat = l_epsinv[0]

# for row in range(0, eps0mat.shape[0]):
#     for col in range(0, eps0mat.shape[1]):
#         print(f"eps0mat[{row}, {col}] = {eps0mat[row, col]}")


In [ ]:
l_epsinv_sorted = kernelclass.l_epsinv
eps0mat_sorted = l_epsinv_sorted[0]

for row in range(0, eps0mat_sorted.shape[0]):
    for col in range(0, eps0mat_sorted.shape[1]):
        print(f"eps0mat_sorted[{row}, {col}] = {eps0mat_sorted[row, col]}")

eps0mat_sorted[0, 0] = (0.007906698375054658-5.293955920339377e-23j)
eps0mat_sorted[0, 1] = (-6.066918913794252e-06-9.943920626670979e-10j)
eps0mat_sorted[0, 2] = (5.766757445574977e-06-4.405815346214877e-12j)
eps0mat_sorted[0, 3] = (-6.068446096038483e-06+1.3787802881480622e-10j)
eps0mat_sorted[0, 4] = (1.4709698783097223e-08-2.6509067339969463e-10j)
eps0mat_sorted[0, 5] = (5.768274803839067e-06-1.3531714810469818e-10j)
eps0mat_sorted[0, 6] = (4.2012229607250794e-08-7.99140931162481e-10j)
eps0mat_sorted[0, 7] = (-6.067919246844843e-06-1.2436375750271802e-10j)
eps0mat_sorted[0, 8] = (1.4491698917861973e-08-6.445045389786882e-10j)
eps0mat_sorted[0, 9] = (1.4785111559309006e-08-5.579863755904097e-10j)
eps0mat_sorted[0, 10] = (1.7664267456498127e-05-7.320718925151357e-10j)
eps0mat_sorted[0, 11] = (5.767536479438961e-06-2.3398913468001786e-10j)
eps0mat_sorted[0, 12] = (4.214222257350715e-08+5.053850639849212e-11j)
eps0mat_sorted[0, 13] = (4.1861698093402334e-08-1.1289875944067249e-11j)
eps